# About
- To simplify the template, this line assumes we are working with local pickled data
- All methods will assume Pandas dataframe is being used
- Other loading examples can be found at the bottom of this template

- For Deep Learning (Object Detection Training), it is best to work with cuda
- As such this template assumes that you are working on cuda device with PyTorch Tensor objects

<!-- TABLE OF CONTENTS -->
<details>
  <summary>Table of Contents</summary>
  <ol>
    <li>
      <a href="#pandas-load">About The Project</a>
    </li>
    <li>
      <a href="#getting-started">Getting Started</a>
    </li>
    <li><a href="#contact">Contact</a></li>
  </ol>
</details>

# Starter Imports


In [ ]:
# Model Building
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.models import detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Data Extraction
from PIL import Image # Image editing
import io
import pandas as pd
import os
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt

# Testing
import time
from tqdm import tqdm

# Load Dataset

## Pandas Load
```python
df_train = pd.read_pickle('PickledDataset.pkl')
```

## HuggingFace Load
```python
data_files = {"train": "train*", "test": "test*"}
ds_train = load_dataset("parquet", data_dir="C:\\Users\\bjorn\\mtg-detection\\data\\", data_files=data_files, split="train[:20%]")
ds_train = ds_train.with_format("torch", device=device)
```

## XML Load
```python

```

# Preprocess Metadata to Determine Labels

In [ ]:
"""
It can be safe and easy to hardcode the number of classes
when the dataset is well regulated and has clear documentation.
Otherwise, it is advised to iterate over your dataset's metadata
in order to confirm the number of classes and to initialize a dictionary
to make class/label encoding easier in future processes.

Remember: The number of classes is equal to the known classes + 1 for background
which signifies "no object". This extra background class should be labeled as 0.

The first step is to iterate over all available data to create a dictionary which maps classes from their original format (String, JSON, etc.)\
to a number encoding. This also allows us to determine our total number of classes which is equal to the known classes + 1 for background\
which signifies "no object". This extra background class should be labeled as 0.

"""
# Create numpy array of all unique labels
unique_classes = df_train['metadata'].unique()

# Create a label dictionary 
label_dict = {}
i = 1 # Start at 1 because 0 is reserved for background objects
for label in unique_classes:
    label_dict[label] = i
    i += 1

number_of_classes = unique_classes.size + 1

# Set Device type

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device) # Double check that you are connected to the desired device

# Model Switch

In [ ]:
# Initialize a dictionary containing torchvision models to simplify testing a variety of training approaches
models = {
	"frcnn-resnet": detection.fasterrcnn_resnet50_fpn,
    "frcnn-mobilenet": detection.fasterrcnn_mobilenet_v3_large_320_fpn,
}

# Load the model and set it to evaluation mode
model = models["frcnn-mobilenet"](weights="DEFAULT").to(device)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, number_of_classes)

# Insert load
model.load_state_dict(torch.load('CardWeights.pt'))

# Helper function to retrieve object data from dataset

In [4]:
# Retrieves labels/bboxes and reformats image data from binary to torch tensors
# Returns target: a dict of bboxes and labels; imgs: tuple of torch tensor images
def generate_target(object):

    # Bounding boxes for objects
    # In pytorch, the input should be [xmin, ymin, xmax, ymax]
    boxes = object['annotation']
    labels = torch.ones(len(object['annotation'])) # All objects are of class "Card" / "1"
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.as_tensor(labels, dtype=torch.int64)

    # Convert image from Byte to Tensor
    img = Image.open(io.BytesIO(object['image'])).convert("RGB")
    data_transform = transforms.Compose([transforms.PILToTensor(), transforms.ConvertImageDtype(torch.float),])
    img = data_transform(img).to(device)

    # Annotation is in dictionary format
    # Innate labels, bboxes and classifier (card vs. background)
    target = {}
    target["boxes"] = boxes.to(device)
    target["labels"] = labels.to(device)

    return target, img

# CREATE DATALOADER

## Create collate function for DataLoader()

In [5]:
def collate_fn(batch):
  return tuple(zip(*batch))

## Create custom map-style dataset to work with pickled pandas dataframe

In [6]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return len(self.df.index)

    def __getitem__(self, idx):
        target, image = generate_target(self.df.loc[idx])
        return target, image

## Create train and validation DataLoader() objects

In [7]:
# Test various batch sizes, higher batch = more memory usage but faster
# If memory usage too high, then model trains slower and batch size must be reduced (idk where this point is)
train_dl = torch.utils.data.DataLoader(dataset=CustomImageDataset(df_train),
                                          batch_size=8,
                                          collate_fn=collate_fn)

# Make Custom Faster RCNN

In [8]:
# Load the pretrained Faster R-CNN model with a MobileNetV3 backbone
model = fasterrcnn_mobilenet_v3_large_320_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)


# Fine-tune Faster R-CNN model on our dataset

## Create optimizer

In [9]:
# You can test other optimizers you find online, this is a standard one I use for Faster RCNN projects
optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.005,
                            momentum=0.9,
                            weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [10]:
# OPTIONAL if training a specific model further
# model.load_state_dict(torch.load('CardWeights.pt'))

## Fine-tune

In [ ]:
model.to(device)
num_epochs = 100
len_dataloader = len(train_dl)
e_num = 1
losses_for_plot = []

for epochs in range(num_epochs):
  i = 1
  model.train()
  epoch_loss = 0
  start = time.time()
  for targets, imgs in tqdm(train_dl):

    imgs = list(img.to(DEVICE) for img in imgs)
    annotations = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

    loss_dict = model(imgs, annotations)
    loss = sum(loss for loss in loss_dict.values())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss

  lr_scheduler.step()
  print(f' Loss: {epoch_loss}, Time: {time.time() - start}, Epoch: {e_num}')
  losses_for_plot.append(epoch_loss.item())
  start = time.time()
  e_num += 1
  torch.save(model.state_dict(),'CardWeights1.pt') # Save after every Epoch
  

# Visualize Loss

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(losses_for_plot)

# Adding labels and title
plt.title("Loss Per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")

# Display the plot
plt.show()

# Additonal Approaches

```python
from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 
df['species']= label_encoder.fit_transform(df['species']) 
df['species'].unique() 
```